In [ ]:
!pip install ultralytics wandb roboflow -q

In [ ]:
from ultralytics import settings

settings.update({"wandb": True})

print(settings['wandb'])

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [ ]:
import wandb

wandb.login(key=user_secrets.get_secret("wandb"))

In [ ]:
project = "PPE Detection Model"

In [ ]:
api = wandb.Api()

runs = api.runs(f'kelpinnnnnn/{project}')

run_ids = []

for run in runs[:10]:
    print(run.id, run.name)
    run_ids.append(run.id)

In [ ]:
for id in run_ids:
    try:
        # Coba download dari project pertama
        artifact = api.artifact(f'kelpinnnnnn/{project}/run_{id}_model:v0', type='model')
        artifact.download()
        print(f"✅ Success: {id} added to run_ids from {project}")
    except Exception as e1:
        print(f"❌ Failed from {project} for run {id}: {e1}")
        try:
            # Coba ulang dari project kedua jika yang pertama gagal
            print("🔁 Trying Ultralytics project...")
            artifact = api.artifact(f'kelpinnnnnn/Ultralytics/run_{id}_model:v0', type='model')
            artifact.download()
            print(f"✅ Success: {id} added to run_ids from Ultralytics")
        except Exception as e2:
            print(f"❌ Failed from Ultralytics for run {id}: {e2}")

In [ ]:
!gdown --fuzzy "https://drive.google.com/file/d/1YUOEHnsuS_1rh6VHgHTKLd3YgoxFJ6TG/view?usp=sharing" -O "foto_jarak.zip"

In [ ]:
!unzip "/kaggle/working/foto_jarak.zip" -d "foto_jarak"

In [ ]:
!rm "/kaggle/working/foto_jarak.zip"

In [ ]:
import os
import shutil

# Direktori sumber
source_dir = '/kaggle/working/foto_jarak/Foto tes'

# Direktori tujuan utama
base_output_dir = 'foto_testing_jarak'

# Pastikan direktori tujuan ada
os.makedirs(base_output_dir, exist_ok=True)

# Proses semua file
for filename in os.listdir(source_dir):
    if filename.endswith('.jpg'):
        lower_name = filename.lower()
        # Ambil nama warna (kuning, biru, merah, putih, rompi)
        if lower_name.startswith('kuning'):
            warna = 'Kuning'
        elif lower_name.startswith('biru'):
            warna = 'Biru'
        elif lower_name.startswith('merah'):
            warna = 'Merah'
        elif lower_name.startswith('putih'):
            warna = 'Putih'
        elif lower_name.startswith('rompi'):
            warna = 'Rompi'
        else:
            continue  # Skip jika tidak sesuai pola

        # Tentukan arah (depan atau samping)
        if 'depan' in lower_name:
            arah = 'depan'
        elif 'samping' in lower_name:
            arah = 'samping'
        else:
            continue  # Skip jika tidak sesuai pola

        # Buat folder tujuan jika belum ada
        target_dir = os.path.join(base_output_dir, warna, arah)
        os.makedirs(target_dir, exist_ok=True)

        # Pindahkan file
        src_path = os.path.join(source_dir, filename)
        dst_path = os.path.join(target_dir, filename)
        shutil.move(src_path, dst_path)
        print(f"Moved: {filename} -> {target_dir}")


In [ ]:
from ultralytics import YOLO  # Ultralytics YOLO
import cv2
from pathlib import Path

# Konfigurasi
input_base_dir = "/kaggle/working/foto_testing_jarak"
output_base_dir = "/kaggle/working/output_deteksi"
warna_list = ["Kuning", "Biru", "Merah", "Putih", "Rompi"]
arah_list = ["depan"]

# Dictionary untuk menyimpan semua hasil
all_results = {}

for id in run_ids:
    run = api.run(f"kelpinnnnnn/{project}/{id}")
    model_path=f"/kaggle/working/artifacts/run_{id}_model:v0"
    # Load the YOLO model
    model = YOLO(f"{model_path}/best.pt")

    model_key = f"{run.name}"
    all_results[model_key] = {}

    for warna in warna_list:
        all_results[model_key][warna] = {}
        
        for arah in arah_list:
            input_dir = Path(input_base_dir) / warna / arah
            all_results[model_key][warna][arah] = {}

            input_dir = os.path.join(input_base_dir, warna, arah)
            output_dir = os.path.join(output_base_dir, f"{run.name}", warna, arah)
            os.makedirs(output_dir, exist_ok=True)

            for filename in os.listdir(input_dir):
                if filename.endswith(".jpg"):
                    img_path = os.path.join(input_dir, filename)

                    # Baca dan resize gambar
                    img = cv2.imread(img_path)
                    resized_img = cv2.resize(img, (640, 640))

                    # Jalankan prediksi
                    results = model.predict(source=resized_img, imgsz=640, conf=0.5)
                    boxes = results[0].boxes

                    # Simpan hasil deteksi per file
                    detections = []
                    for box in boxes:
                        xyxy = box.xyxy[0].tolist()       # [x1, y1, x2, y2]
                        conf = float(box.conf[0])         # confidence
                        cls = int(box.cls[0])             # class id
                        detections.append({
                            "bbox": xyxy,
                            "confidence": conf,
                            "class_id": cls
                        })

                    all_results[model_key][warna][arah][filename] = detections

                    # Simpan hasil deteksi
                    save_path = os.path.join(output_dir, filename)
                    results[0].save(filename=save_path)

                    print(f"✅ Model {run.name} - {warna}/{arah}/{filename} processed.")




In [ ]:
import json

# Simpan seluruh hasil ke file JSON
output_json_path = "/kaggle/working/output_deteksi/results_all_models.json"
with open(output_json_path, "w") as f:
    json.dump(all_results, f, indent=2)

print(f"✅ Semua hasil deteksi disimpan ke: {output_json_path}")

In [ ]:
!zip -r "output_deteksi_hasil_deteksi.zip" "output_hasil_deteksi"